# Решение финансовых ЗЛП с помощью симплекс метода
## Постановка задачи
$x_1, x_2, x_3$ - количество счетов физ. лицам, юр. лицам и индивидуальным предпринимателям.

Доходность услуг для банка:
$F(x_1,x_2,x_3) = 5  x_1 + 6  x_2 + 7  x_3 \to \max$ 


Ограничения: 
1. Финансовые затраты на для ведения счета
2. Затраты ресурсов сотрудников для ведения счета

$\begin{cases} 
1.2 x_1 + 0.8 x_2 + 0.6 x_3 \leq 500 \\
0.5 x_1 + 0.7 x_2 + 1 x_3 \leq 400 \\
x_1, x_2, x_3 \geq 0
\end{cases}$

## Разработка программы

1. Функции чтения файла
2. Функция приведения к канонической форме
3. Шаг симплекс метода
4. Вспомогательный шаг
5. Запуск симплекс метода 
6. Интеграция с ChatGpt для автоматизации формирования математической постановки

In [567]:
import json
import numpy as np

def read_file(name):
    with open(name, 'r') as f:
        data = f.read()

    return parse_input(data)


def parse_input(data):
    data = json.loads(data)
    if data['direction'] == 'min':
        c = np.array(data['W'], dtype=np.float64)
    else:
        c = - np.array(data['W'], dtype=np.float64)
    A = np.array(data['A'])
    b = np.array(data['b'])
    constr_dir = data['constraints_direction']
    unique, counts = np.unique(constr_dir, return_counts=True)
    c_dict = dict(zip(unique, counts))
    return data, A, b, c, constr_dir, c_dict

In [568]:
def make_canonical_form(data, A, constr_dir, c, c_dict):    
    A = np.hstack([A, np.zeros((len(A), c_dict['l']))])
    
    j = len(c)
    for i in range(len(constr_dir)):
        if data['constraints_direction'][i] == 'g':
            A[i, j] = -1
            j += 1
                
        elif data['constraints_direction'][i] == 'l':
            A[i, j] = 1
            j += 1
            
        elif data['constraints_direction'][i] == 'e':
            continue
        
        else:
            print('Wrong constraint direction')
            
    return A

In [569]:
data, A, b, c, constr_dir, c_dict = read_file('simplex_task.txt')
A = make_canonical_form(data, A, constr_dir, c, c_dict)
print(A)

[[1.2 0.8 0.6 1.  0. ]
 [0.5 0.7 1.  0.  1. ]]


In [570]:
import copy

def simplex_step(A_s, b_s, z, Q, base_vars, free_vars):
    main_col = np.argmin(z)
    
    
    penalty = 100_000
    row_array = []
    for i in range(len(A_s)):
        if A_s[i, main_col] > 0:
            row_array.append(b_s[i] / A_s[i, main_col])
        else:
            row_array.append(penalty)
            
    if np.sum(row_array) == penalty * len(A_s):
        print('Целевая функция не ограничена снизу')
        return 0
            
    else:
        main_row = np.argmin(row_array)
        main_el = A_s[main_row, main_col]
        
        
        #compute new matrix A 
        A_s_new = copy.deepcopy(A_s)
        A_s_new[main_row] = A_s[main_row] / main_el
        A_s_new[:, main_col] = -A_s[:, main_col]/main_el
        A_s_new[main_row, main_col] = 1/main_el
        
        for i in range(A_s.shape[0]):
            for j in range(A_s.shape[1]):
                if (i != main_row) and (j != main_col):
                    A_s_new[i, j] = A_s[i, j] - A_s[main_row, j] * A_s[i, main_col] / main_el
        
        
        #compute new z vector
        z_new = copy.deepcopy(z)
        z_new[main_col] = -z[main_col]/main_el
        for j in range(len(z_new)):
            if j != main_col:
                z_new[j] = z[j] - z[main_col] * A_s[main_row, j] / main_el
        
        
        #compute new b vector
        b_s_new = copy.deepcopy(b_s)
        b_s_new[main_row] = b_s[main_row]/main_el
        for i in range(len(b_s_new)):
            if i != main_row:
                b_s_new[i] = b_s[i] - b_s[main_row] * A_s[i, main_col] / main_el
        
        #compute cost function
        Q_new = Q - b_s[main_row] * z[main_col] / main_el
        
        #change base vars with free vars
        change_val = base_vars[main_row]
        base_vars[main_row] = free_vars[main_col]
        free_vars[main_col] = change_val
        
        
        return A_s_new, b_s_new, z_new, Q_new, base_vars, free_vars

In [571]:
def auxilary_task_step(A_s, b_s, z, Q, base_vars, free_vars):
    main_col = np.argmin(z)
    
    penalty = 100_000
    row_array = []
    for i in range(len(A_s)):
        if A_s[i, main_col] > 0:
            row_array.append(b_s[i] / A_s[i, main_col])
        else:
            row_array.append(penalty)
            
    if np.sum(row_array) == penalty * len(A_s):
        print('Unconstrained objective function')
        return 0
            
    else:
        main_row = np.argmin(row_array)
        main_el = A_s[main_row, main_col]
        
        
        #compute new matrix A 
        A_s_new = copy.deepcopy(A_s)
        A_s_new[main_row] = A_s[main_row] / main_el
        A_s_new[:, main_col] = -A_s[:, main_col]/main_el
        A_s_new[main_row, main_col] = 1/main_el
        
        for i in range(A_s.shape[0]):
            for j in range(A_s.shape[1]-1):
                if (i != main_row) and (j != main_col):
                    A_s_new[i, j] = A_s[i, j] - A_s[main_row, j] * A_s[i, main_col] / main_el
        
        
        #compute new z vector
        z_new = copy.deepcopy(z)
        z_new[main_col] = -z[main_col]/main_el
        for j in range(len(z_new)):
            if j != main_col:
                z_new[j] = z[j] - z[main_col] * A_s[main_row, j] / main_el
        
        
        #compute new b vector
        b_s_new = copy.deepcopy(b_s)
        b_s_new[main_row] = b_s[main_row]/main_el
        for i in range(len(b_s_new)):
            if i != main_row:
                b_s_new[i] = b_s[i] - b_s[main_row] * A_s[i, main_col] / main_el
        
        #compute cost function
        Q_new = Q - b_s[main_row] * z[main_col] / main_el
        
        #change base vars with free vars
        change_val = base_vars[main_row]
        base_vars[main_row] = free_vars[main_col]
        free_vars[main_col] = change_val

        #delete main column with base variable
        free_vars = np.delete(free_vars, main_col)
        A_s_new = np.delete(A_s_new, main_col, axis=1)
        z_new = np.delete(z_new, main_col)
        
        return A_s_new, b_s_new, z_new, Q_new, base_vars, free_vars

In [572]:
# b_s = b
# A_s = A
# z = -np.sum(A_s, 0)
# Q = -np.sum(b_s, 0)
# 
# base_vars = np.arange(len(A[0]) + 1, len(A[0]) + 1 + len(b))
# free_vars = np.arange(1, len(A[0]) + 1)

In [573]:
def pretty_print(A_s, b_s, z, Q, base_vars, free_vars):
    print("---Begin---")
    print("A_s:")
    for row in A_s:
        print("  ", " ".join(f"{val:8.2f}" for val in row))
    
    print("\nb_s:")
    print("  ", " ".join(f"{val:8.2f}" for val in b_s))
    
    print("\nz:")
    print("  ", " ".join(f"{val:8.2f}" for val in z))
    
    print("\nQ:")
    print("  ", f"{Q:8.2f}")
    
    print("\nBase Variables:")
    print("  ", " ".join(map(str, base_vars)))
    
    print("\nFree Variables:")
    print("  ", " ".join(map(str, free_vars)))
    print("---End---")

In [574]:
# pretty_print(A_s, b_s, z, Q, base_vars, free_vars)
# while np.any(z < 0):
#     A_s, b_s, z, Q, base_vars, free_vars = auxilary_task_step(A_s, b_s, z, Q, base_vars, free_vars)
#     pretty_print(A_s, b_s, z, Q, base_vars, free_vars)

In [575]:
b_s = b
A_s = A[:, :-2]
z = c
Q = 0
base_vars = np.arange(len(A_s[0]) + 1, len(A_s[0]) + 1 + len(b))
free_vars = np.arange(1, len(A_s[0]) + 1)

In [576]:
pretty_print(A_s, b_s, z, Q, base_vars, free_vars)

---Begin---
A_s:
       1.20     0.80     0.60
       0.50     0.70     1.00

b_s:
     500.00   400.00

z:
      -5.00    -6.00    -7.00

Q:
       0.00

Base Variables:
   4 5

Free Variables:
   1 2 3
---End---


In [577]:
while np.any(z < 0):
    A_s, b_s, z, Q, base_vars, free_vars = simplex_step(A_s, b_s, z, Q, base_vars, free_vars)
    pretty_print(A_s, b_s, z, Q, base_vars, free_vars)

---Begin---
A_s:
       0.90     0.38    -0.60
       0.50     0.70     1.00

b_s:
     260.00   400.00

z:
      -1.50    -1.10     7.00

Q:
    2800.00

Base Variables:
   4 3

Free Variables:
   1 2 5
---End---
---Begin---
A_s:
       1.11     0.42    -0.67
      -0.56     0.49     1.33

b_s:
     288.89   255.56

z:
       1.67    -0.47     6.00

Q:
    3233.33

Base Variables:
   1 3

Free Variables:
   4 2 5
---End---
---Begin---
A_s:
       1.59    -0.86    -1.82
      -1.14     2.05     2.73

b_s:
      68.18   522.73

z:
       1.14     0.95     7.27

Q:
    3477.27

Base Variables:
   1 2

Free Variables:
   4 3 5
---End---


In [578]:
result = sorted(zip(base_vars, b_s), key=lambda x: x[0])
print(result)
result = [(key, value) for key, value in result if key <= 3]
result

[(1, 68.18181818181813), (2, 522.7272727272727)]


[(1, 68.18181818181813), (2, 522.7272727272727)]